# Description
This notebook will help setup and run a HAPI FHIR server, and load synthetic data into it.

# Requirements
* Python with the `phdi` package installed
* Java 11+ JDK - not JRE (for Synthea)
* (recommended) wget command line tool

In [ ]:
import os
import requests
import subprocess

synthea_url = "https://github.com/synthetichealth/synthea/releases/download/master-branch-latest/synthea-with-dependencies.jar"


# Download the synthea jar file
synthea_jar_response = requests.get(synthea_url, stream=True)
if not os.path.isdir("target"):
    os.mkdir("target")
if not os.path.exists("target/synthea-with-dependencies.jar"):
    print("Downloading synthea-with-dependencies.jar...")
    with open("target/synthea-with-dependencies.jar","wb") as file:
        for chunk in synthea_jar_response.iter_content(chunk_size=1024):
            file.write(chunk)

# Create Synthea data
subprocess.run(args=["java", "-jar", "synthea-with-dependencies.jar", "-p", "1000"], cwd="target")

The command above should have generated 1000 syntentic patients in FHIR format.  Now we need to upload them to the FHIR server.

In [6]:
import json
import phdi.fhir.transport
import os
import pathlib
from phdi.fhir.transport.http import (
    http_request_with_reauth
)

base_path = os.path.join("target","output","fhir")
for fhir_filename in os.listdir(base_path):
    fhir_filepath = os.path.join(base_path, fhir_filename)
    print(f"Evaluating {fhir_filepath}")

    if os.path.isfile(fhir_filepath) and fhir_filepath.endswith(".json"):
        with open(fhir_filepath) as fhir_file:
            print(f"Importing {fhir_filepath}")
            http_request_with_reauth(cred_manager=None, url="http://localhost:8080/", retry_count=3, request_type="POST", allowed_methods="POST", headers={}, data=json.load(fhir_file))

Evaluating target/output/fhir/Lourdes258_Emelina469_Kozey370_88713f12-0afd-41db-9467-2c3bf8f740e8.json
Importing target/output/fhir/Lourdes258_Emelina469_Kozey370_88713f12-0afd-41db-9467-2c3bf8f740e8.json


AttributeError: module 'phdi' has no attribute 'fhir'